# Download Raw Data

Let's create a function to download one file of raw data from NYC data from https://www.nyc.gov/site/tlc/about/tlc-trip-record-data.page

Here we mainly do three steps:

1. Download the raw data with the function `download_one_file_of_raw_data`
2. Validate the raw downloaded data by filtering it. Function `validate_raw_data`
3. Save the downloaded data on the `data/raw` folder

Each of this function are going to be placed inside a module file: `src/data.py`

In [12]:
import os
import requests
from pathlib import Path

# src/data.py
def download_one_file_of_raw_data(year: int, month: int) -> Path:

    URL = f"https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_{year}-{month:02d}.parquet"
    response = requests.get(URL)

    if response.status_code == 200:
        os.makedirs("../data/raw", exist_ok=True)
        path = f"../data/raw/rides_{year}-{month:02d}.parquet"
        with open(path, 'wb') as f:
            f.write(response.content)
        return path
    else:
        raise Exception(f"{URL} is not available")

In [13]:
download_one_file_of_raw_data(year=2022, month=1)

'../data/raw/rides_2022-01.parquet'

Loading the downloaded data

In [14]:
import pandas as pd

rides = pd.read_parquet("../data/raw/rides_2022-01.parquet")

rides.head()

,VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,RatecodeID,store_and_fwd_flag,PULocationID,DOLocationID,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,congestion_surcharge,airport_fee
0,1,2022-01-01 00:35:40,2022-01-01 00:53:29,2.0,3.80,1.0,N,142,236,1,14.5,3.0,0.5,3.65,0.0,0.3,21.95,2.5,0.0
1,1,2022-01-01 00:33:43,2022-01-01 00:42:07,1.0,2.10,1.0,N,236,42,1,8.0,0.5,0.5,4.00,0.0,0.3,13.30,0.0,0.0
2,2,2022-01-01 00:53:21,2022-01-01 01:02:19,1.0,0.97,1.0,N,166,166,1,7.5,0.5,0.5,1.76,0.0,0.3,10.56,0.0,0.0
3,2,2022-01-01 00:25:21,2022-01-01 00:35:23,1.0,1.09,1.0,N,114,68,2,8.0,0.5,0.5,0.00,0.0,0.3,11.80,2.5,0.0
4,2,2022-01-01 00:36:48,2022-01-01 01:14:20,1.0,4.30,1.0,N,68,163,1,23.5,0.5,0.5,3.00,0.0,0.3,30.30,2.5,0.0


Picking only the needed columns

In [15]:
rides = rides[['tpep_pickup_datetime', 'PULocationID']].copy()

Renaming the columns

In [16]:
rides = rides.rename(columns={
    'tpep_pickup_datetime': 'pickup_datetime',
    'PULocationID': 'pickup_location_id'
})

rides.head()

,pickup_datetime,pickup_location_id
0,2022-01-01 00:35:40,142
1,2022-01-01 00:33:43,236
2,2022-01-01 00:53:21,166
3,2022-01-01 00:25:21,114
4,2022-01-01 00:36:48,68


In [ ]:
# src/data.py
def select_and_rename_raw_columns(rides: pd.DataFrame) -> pd.DataFrame:
    """
    Selects and renames columns from the raw data
    """
    rides = rides[['tpep_pickup_datetime', 'PULocationID']].copy()
    rides.rename(columns={
        'tpep_pickup_datetime': 'pickup_datetime',
        'PULocationID': 'pickup_location_id',
    }, inplace=True)

    return rides

The data downloaded has some data from outside January 2022, and it shouldn't. Let's validate it by filtering only rides from January 2022.

In [17]:
rides['pickup_datetime'].describe()

count                          2463931
mean     2022-01-17 01:19:51.689726208
min                2008-12-31 22:23:09
25%                2022-01-09 15:37:41
50%                2022-01-17 12:11:45
75%         2022-01-24 13:49:37.500000
max                2022-05-18 20:41:57
Name: pickup_datetime, dtype: object

In [18]:
# validating raw data by filtering it
rides = rides[rides['pickup_datetime'] >= '2022-01-01']
rides = rides[rides['pickup_datetime'] < '2022-02-01']
rides['pickup_datetime'].describe()

count                          2463879
mean     2022-01-17 01:58:40.393673472
min                2022-01-01 00:00:08
25%                2022-01-09 15:37:56
50%                2022-01-17 12:11:54
75%                2022-01-24 13:49:37
max                2022-01-31 23:59:58
Name: pickup_datetime, dtype: object

In [21]:
# src/data.py
def validate_raw_data(
    rides: pd.DataFrame,
    year: int,
    month: int,
) -> pd.DataFrame:
    """
    Removes rows with pickup_datetimes outside their valid range
    """
    # keep only rides for this month
    this_month_start = f'{year}-{month:02d}-01'
    next_month_start = f'{year}-{month+1:02d}-01' if month < 12 else f'{year+1}-01-01'
    rides = rides[rides.pickup_datetime >= this_month_start]
    rides = rides[rides.pickup_datetime < next_month_start]
    
    return rides

In [23]:
validate_raw_data(rides, year=2022, month=1).describe()

,pickup_datetime,pickup_location_id
count,2463879,2.463879e+06
mean,2022-01-17 01:58:40.393673472,1.660769e+02
min,2022-01-01 00:00:08,1.000000e+00
25%,2022-01-09 15:37:56,1.320000e+02
50%,2022-01-17 12:11:54,1.620000e+02
75%,2022-01-24 13:49:37,2.340000e+02
max,2022-01-31 23:59:58,2.650000e+02
std,NaN,6.546809e+01


In [24]:
os.makedirs('../data/transformed', exist_ok=True)
rides.to_parquet('../data/transformed/validated_rides_2022_01.parquet')

# Function to encapsulate all the steps

First, let's create a file `src/paths.py` and put inside it the following content:

```python
from pathlib import Path
import os

PARENT_DIR = Path(__file__).parent.resolve().parent
DATA_DIR = PARENT_DIR / 'data'
RAW_DATA_DIR = PARENT_DIR / 'data' / 'raw'

if not Path(DATA_DIR).exists():
    os.mkdir(DATA_DIR)

if not Path(RAW_DATA_DIR).exists():
    os.mkdir(RAW_DATA_DIR)
```

In [ ]:
from typing import Optional, List
from src.paths import RAW_DATA_DIR

# src/data.py
def load_raw_data(
    year: int,
    months: Optional[List[int]] = None
) -> pd.DataFrame:
    """
    Loads raw data from local storage or downloads it from the NYC website, and
    then loads it into a Pandas DataFrame

    Args:
        year: year of the data to download
        months: months of the data to download. If `None`, download all months

    Returns:
        pd.DataFrame: DataFrame with the following columns:
            - pickup_datetime: datetime of the pickup
            - pickup_location_id: ID of the pickup location
    """  
    rides = pd.DataFrame()
    
    if months is None:
        # download data for the entire year (all months)
        months = list(range(1, 13))
    elif isinstance(months, int):
        # download data only for the month specified by the int `month`
        months = [months]

    for month in months:
        
        local_file = RAW_DATA_DIR / f'rides_{year}-{month:02d}.parquet'
        if not local_file.exists():
            try:
                # download the file from the NYC website
                print(f'Downloading file {year}-{month:02d}')
                download_one_file_of_raw_data(year, month)
            except:
                print(f'{year}-{month:02d} file is not available')
                continue
        else:
            print(f'File {year}-{month:02d} was already in local storage') 

        # load the file into Pandas
        rides_one_month = pd.read_parquet(local_file)

        # rename columns
        rides_one_month = select_and_rename_raw_columns(rides_one_month)

        # validate the file
        rides_one_month = validate_raw_data(rides_one_month, year, month)

        # append to existing data
        rides = pd.concat([rides, rides_one_month])

    if rides.empty:
        # no data, so we return an empty dataframe
        return pd.DataFrame()
    else:
        # keep only time and origin of the ride
        rides = rides[['pickup_datetime', 'pickup_location_id']]
        return rides

# Exercise

1. Put the three functions created here inside a file called `src/data.py`
2. Create a notebook `notebooks/01b_load_and_validate_raw_data_functions.ipynb` and test the functions created